# Prediciting high school graduation rate of a cohort based on holistic data of the student population and school.

What do variables about:
- the broader structure of a school
- the experience of previous cohorts
- the freshmen, sophomore and junior year experience of rising seniors
tell us about the graduating % of rising seniors?

Data was gathered from https://www.isbe.net/pages/illinois-state-report-card-data.aspx
I have collected end of year data for the following academic years:
- 2018/19
- 2017/18
- 2016/17
- 2015/16
- 2014/15
- 2013/14
- 2012/13

I will look at the following cohorts for training the data:

- Class of 2017
- Class of 2016

and follow them thru their 4 years of high school (grades 9-12)

I will run validation on 
- Class of 2018

My test set with be 
- Class of 2019

The goal is to predict the 4-year high school graduation rate for this cohort.

# Sections
[Cleaning and exploring datasets](#Cleaning-and-exploring-datasets)
- [Class of 2019](#Class-of-2019)
- [Class of 2018](#Class-of-2018)
- [Class of 2017](#Class-of-2017)
- [Class of 2016](#Class-of-2016)
- [Class of 2015](#Class-of-2015)
- [Class of 2014](#Class-of-2014)
- [Class of 2013](#Class-of-2013)

[Concatenate dataframes](#Concatenate-dataframes)

[EDA on merged dataframe](#EDA-on-merged-dataframe)




## Cleaning and exploring datasets

Gathering the data for each of the academic years 2013 - 2017 involved downloading semi-colon separated .txt files, importing them into a spreadsheet, then matching the columns to headers that were recorded in a separate .xls file. While performing this task, I also selected feature set to use for this project.

I also added a column to every academic year to identify the cohort by their graduation year (e.g. 2018). 

I will create a matrix of the information for each cohort.

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# import standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

# import additional libraries
import pandasql as ps
from pandasql import sqldf

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

# functions from .py file
import src.eda_functions as fun

# pysqldf lambda function
pysqldf = lambda q: sqldf(q, globals())

# turn off warnings
import warnings
warnings.simplefilter('ignore', category = DeprecationWarning)
warnings.simplefilter('ignore', category = FutureWarning)


/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# plot parameters
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 25
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['axes.edgecolor'] = 'black'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['font.size'] = 16

### Step 1
create a dataframe for each academic year for schools that serve grades 9 - 12.
This will include traditional high schools and some charter schools that serve grades in addition to 9 - 12. This may have an impact on features such as the total number of students in a school, attendance rate and I will have to be aware and make corrections if needed.

### Class of 2019

In [4]:
df_19 = pd.read_excel('../../data/2019-Report-Card-Public-Data-Set_clean.xlsx', sheet_name="General")
df_19.shape

(4738, 35)

In [5]:
fun.clean_col(df_19)

['cohort',
 'rcdts',
 'school_name',
 'district',
 'city',
 'county',
 'district_type',
 'district_size',
 'school_type',
 'grades_served',
 'percent_student_enrollment_white',
 'percent_student_enrollment_black_or_african_american',
 'percent_student_enrollment_hispanic_or_latino',
 'percent_student_enrollment_asian',
 'percent_student_enrollment_native_hawaiian_or_other_pacific_islander',
 'percent_student_enrollment_american_indian_or_alaska_native',
 'percent_student_enrollment_two_or_more_races',
 'number_student_enrollment',
 'total_number_of_school_days',
 'student_attendance_rate',
 'student_chronic_truancy_rate',
 'high_school_dropout_rate_total',
 'high_school_4_year_graduation_rate_total',
 'high_school_5_year_graduation_rate_total',
 'avg_class_size_high_school',
 'pupil_teacher_ratio_high_school',
 'teacher_avg_salary',
 'teacher_retention_rate',
 'principal_turnover_within_6_years',
 'percent_graduates_enrolled_in_a_postsecondary_institution_within_16_months',
 'percent_g

In [6]:
# create query to return schools with grades 9-12

q1 = """SELECT * 
       FROM df_19
       WHERE grades_served LIKE "%12%";"""

hs_19 = pysqldf(q1)

In [7]:
hs_19.head()

,cohort,rcdts,school_name,district,city,county,district_type,district_size,school_type,grades_served,...,pupil_teacher_ratio_high_school,teacher_avg_salary,teacher_retention_rate,principal_turnover_within_6_years,percent_graduates_enrolled_in_a_postsecondary_institution_within_16_months,percent_graduates_enrolled_in_a_postsecondary_institution_within_12_months,percent_9th_grade_on_track,number_students_who_took_ap_classes_grade_10_total,number_students_who_took_ap_classes_grade_11_total,number_students_who_took_ap_classes_grade_12_total
0,2019,010010010260001,Seymour High School,Payson CUSD 1,Payson,Adams,UNIT,MEDIUM,HIGH SCHOOL,7 8 9 10 11 12,...,None,None,86.8,2.0,68.8,68.8,95.6,NaN,NaN,NaN
1,2019,010010020260001,Liberty High School,Liberty CUSD 2,Liberty,Adams,UNIT,MEDIUM,HIGH SCHOOL,7 8 9 10 11 12,...,None,None,83.9,2.0,58.1,58.1,94.0,NaN,NaN,NaN
2,2019,010010030260001,Central High School,Central CUSD 3,Camp Point,Adams,UNIT,MEDIUM,HIGH SCHOOL,9 10 11 12,...,None,None,83.3,2.0,75.0,73.3,94.9,NaN,NaN,NaN
3,2019,010010040260001,Unity High School,CUSD 4,Mendon,Adams,UNIT,MEDIUM,HIGH SCHOOL,9 10 11 12,...,None,None,86.4,1.0,60.7,60.7,92.9,NaN,NaN,27.0
4,2019,010011720220003,Quincy Sr High School,Quincy SD 172,Quincy,Adams,UNIT,LARGE,HIGH SCHOOL,9 10 11 12,...,None,None,86.4,2.0,67.7,65.7,74.9,68.0,127.0,126.0


In [8]:
hs_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 35 columns):
 #   Column                                                                      Non-Null Count  Dtype  
---  ------                                                                      --------------  -----  
 0   cohort                                                                      720 non-null    int64  
 1   rcdts                                                                       720 non-null    object 
 2   school_name                                                                 720 non-null    object 
 3   district                                                                    720 non-null    object 
 4   city                                                                        720 non-null    object 
 5   county                                                                      720 non-null    object 
 6   district_type                                     

There are some columns that are have nulls. I am going to have to fix those.

the `percentage_student_enrollment`... nulls can be changed to zero.
I am assuming the same can be applied to columns 31-34. 

Will need to investigate further to see why there are missing values in 18-30.

In [9]:
hs_19.percent_student_enrollment_white.isna().sum()

38

In [10]:
hs_19.percent_student_enrollment_white.sort_values(na_position='first')

191      NaN
196      NaN
197      NaN
199      NaN
206      NaN
       ...  
159    100.0
179    100.0
454    100.0
166    100.0
422    100.0
Name: percent_student_enrollment_white, Length: 720, dtype: float64

In [11]:
hs_19.percent_student_enrollment_white.replace(np.nan, 0, inplace=True)

In [12]:
hs_19.percent_student_enrollment_white.isna().sum()

0

OK. that did the trick. Time to do the same for the other enrollment columns

In [13]:
hs_19.percent_student_enrollment_black_or_african_american.replace(np.nan, 0, inplace=True)
hs_19.percent_student_enrollment_hispanic_or_latino.replace(np.nan, 0, inplace=True)
hs_19.percent_student_enrollment_asian.replace(np.nan, 0, inplace=True)
hs_19.percent_student_enrollment_native_hawaiian_or_other_pacific_islander.replace(np.nan, 0, inplace=True)
hs_19.percent_student_enrollment_american_indian_or_alaska_native.replace(np.nan, 0, inplace=True)
hs_19.percent_student_enrollment_two_or_more_races.replace(np.nan, 0, inplace=True)

...and the AP classes columns

In [14]:
hs_19.number_students_who_took_ap_classes_grade_10_total.replace(np.nan, 0, inplace=True)
hs_19.number_students_who_took_ap_classes_grade_11_total.replace(np.nan, 0, inplace=True)
hs_19.number_students_who_took_ap_classes_grade_12_total.replace(np.nan, 0, inplace=True)

In [15]:
hs_19.isnull().sum().sort_values(ascending=False)

pupil_teacher_ratio_high_school                                               720
teacher_avg_salary                                                            720
percent_graduates_enrolled_in_a_postsecondary_institution_within_12_months     58
percent_graduates_enrolled_in_a_postsecondary_institution_within_16_months     58
percent_9th_grade_on_track                                                     38
student_chronic_truancy_rate                                                   14
student_attendance_rate                                                        12
teacher_retention_rate                                                         12
principal_turnover_within_6_years                                               9
avg_class_size_high_school                                                      9
high_school_5_year_graduation_rate_total                                        6
high_school_4_year_graduation_rate_total                                        5
total_number_of_

In [16]:
# cohort should be a category
hs_19['cohort'] = hs_19['cohort'].astype('object')
# check that again
list(hs_19.select_dtypes(['object', 'bool']))

['cohort',
 'rcdts',
 'school_name',
 'district',
 'city',
 'county',
 'district_type',
 'district_size',
 'school_type',
 'grades_served',
 'pupil_teacher_ratio_high_school',
 'teacher_avg_salary']

In [17]:
hs_19 = hs_19.drop(['teacher_avg_salary',
                    'principal_turnover_within_6_years'], axis=1).copy()

In [18]:
zeros = hs_19.loc[:, ('high_school_4_year_graduation_rate_total',
                        'high_school_5_year_graduation_rate_total',
                        'percent_graduates_enrolled_in_a_postsecondary_institution_within_16_months',
                        'percent_graduates_enrolled_in_a_postsecondary_institution_within_12_months',
                        'percent_9th_grade_on_track')]

for zero in zeros:
    hs_19[zero].fillna(0, inplace=True)
    
hs_19.isnull().sum().sort_values(ascending=False)

pupil_teacher_ratio_high_school                                               720
student_chronic_truancy_rate                                                   14
teacher_retention_rate                                                         12
student_attendance_rate                                                        12
avg_class_size_high_school                                                      9
total_number_of_school_days                                                     3
high_school_dropout_rate_total                                                  1
number_students_who_took_ap_classes_grade_12_total                              0
percent_student_enrollment_white                                                0
grades_served                                                                   0
school_type                                                                     0
district_size                                                                   0
district_type   

In [19]:
nans = hs_19.loc[:, ('teacher_retention_rate', 
                    'pupil_teacher_ratio_high_school', 
                    'avg_class_size_high_school', 
                    'high_school_dropout_rate_total', 
                    'student_chronic_truancy_rate',
                    'student_attendance_rate', 
                    'total_number_of_school_days' )]

for nan in nans:
    hs_19[nan].fillna(hs_19[nan].mean(), inplace=True)

In [20]:
hs_19.isnull().sum().sort_values(ascending=False)

pupil_teacher_ratio_high_school                                               720
number_students_who_took_ap_classes_grade_12_total                              0
district_size                                                                   0
percent_student_enrollment_asian                                                0
percent_student_enrollment_hispanic_or_latino                                   0
percent_student_enrollment_black_or_african_american                            0
percent_student_enrollment_white                                                0
grades_served                                                                   0
school_type                                                                     0
district_type                                                                   0
percent_student_enrollment_american_indian_or_alaska_native                     0
county                                                                          0
city            

In [21]:
hs_19.to_csv('../../data/test_set.csv', index=False)

### Class of 2018

In [22]:
df_18 = pd.read_excel('../../data/2018-Report-Card-Public-Data-Set_clean.xlsx', sheet_name="General")
df_18.shape

(4754, 35)

In [23]:
fun.clean_col(df_18)

['cohort',
 'rcdts',
 'school_name',
 'district',
 'city',
 'county',
 'district_type',
 'district_size',
 'school_type',
 'grades_served',
 'percent_student_enrollment_white',
 'percent_student_enrollment_black_or_african_american',
 'percent_student_enrollment_hispanic_or_latino',
 'percent_student_enrollment_asian',
 'percent_student_enrollment_native_hawaiian_or_other_pacific_islander',
 'percent_student_enrollment_american_indian_or_alaska_native',
 'percent_student_enrollment_two_or_more_races',
 'number_student_enrollment',
 'total_number_of_school_days',
 'student_attendance_rate',
 'student_chronic_truancy_rate',
 'high_school_dropout_rate_total',
 'high_school_4_year_graduation_rate_total',
 'high_school_5_year_graduation_rate_total',
 'avg_class_size_high_school',
 'pupil_teacher_ratio_high_school',
 'teacher_avg_salary',
 'teacher_retention_rate',
 'principal_turnover_within_6_years',
 'percent_graduates_enrolled_in_a_postsecondary_institution_within_16_months',
 'percent_g

In [24]:
q2 = """SELECT * 
       FROM df_18
       WHERE grades_served LIKE "%12%";"""

hs_18 = pysqldf(q2)

In [25]:
hs_18.head()

,cohort,rcdts,school_name,district,city,county,district_type,district_size,school_type,grades_served,...,pupil_teacher_ratio_high_school,teacher_avg_salary,teacher_retention_rate,principal_turnover_within_6_years,percent_graduates_enrolled_in_a_postsecondary_institution_within_16_months,percent_graduates_enrolled_in_a_postsecondary_institution_within_12_months,percent_9th_grade_on_track,number_students_who_took_ap_classes_grade_10_total,number_students_who_took_ap_classes_grade_11_total,number_students_who_took_ap_classes_grade_12_total
0,2018,010010010260001,Seymour High School,Payson CUSD 1,Payson,Adams,UNIT,MEDIUM,HIGH SCHOOL,7 8 9 10 11 12,...,None,None,90.6,2.0,81.5,81.5,100.0,NaN,NaN,NaN
1,2018,010010020260001,Liberty High School,Liberty CUSD 2,Liberty,Adams,UNIT,MEDIUM,HIGH SCHOOL,7 8 9 10 11 12,...,None,None,84.8,2.0,80.4,80.4,93.0,NaN,NaN,NaN
2,2018,010010030260001,Central High School,Central CUSD 3,Camp Point,Adams,UNIT,MEDIUM,HIGH SCHOOL,9 10 11 12,...,None,None,83.7,1.0,75.0,75.0,94.0,NaN,NaN,NaN
3,2018,010010040260001,Unity High School,CUSD 4,Mendon,Adams,UNIT,MEDIUM,HIGH SCHOOL,9 10 11 12,...,None,None,83.7,1.0,74.0,72.0,80.0,NaN,NaN,20.0
4,2018,010011720220003,Quincy Sr High School,Quincy SD 172,Quincy,Adams,UNIT,LARGE,HIGH SCHOOL,9 10 11 12,...,None,None,81.9,3.0,68.9,68.2,51.0,109.0,110.0,148.0


In [26]:
hs_18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721 entries, 0 to 720
Data columns (total 35 columns):
 #   Column                                                                      Non-Null Count  Dtype  
---  ------                                                                      --------------  -----  
 0   cohort                                                                      721 non-null    int64  
 1   rcdts                                                                       721 non-null    object 
 2   school_name                                                                 721 non-null    object 
 3   district                                                                    721 non-null    object 
 4   city                                                                        721 non-null    object 
 5   county                                                                      721 non-null    object 
 6   district_type                                     

Going to do the same clean up on hs_18

In [27]:
hs_18.percent_student_enrollment_white.replace(np.nan, 0, inplace=True)
hs_18.percent_student_enrollment_black_or_african_american.replace(np.nan, 0, inplace=True)
hs_18.percent_student_enrollment_hispanic_or_latino.replace(np.nan, 0, inplace=True)
hs_18.percent_student_enrollment_asian.replace(np.nan, 0, inplace=True)
hs_18.percent_student_enrollment_native_hawaiian_or_other_pacific_islander.replace(np.nan, 0, inplace=True)
hs_18.percent_student_enrollment_american_indian_or_alaska_native.replace(np.nan, 0, inplace=True)
hs_18.percent_student_enrollment_two_or_more_races.replace(np.nan, 0, inplace=True)
hs_18.number_students_who_took_ap_classes_grade_10_total.replace(np.nan, 0, inplace=True)
hs_18.number_students_who_took_ap_classes_grade_11_total.replace(np.nan, 0, inplace=True)
hs_18.number_students_who_took_ap_classes_grade_12_total.replace(np.nan, 0, inplace=True)

In [28]:
hs_18.isnull().sum().sort_values(ascending=False)

pupil_teacher_ratio_high_school                                               721
teacher_avg_salary                                                            721
teacher_retention_rate                                                         67
percent_graduates_enrolled_in_a_postsecondary_institution_within_12_months     63
percent_graduates_enrolled_in_a_postsecondary_institution_within_16_months     63
percent_9th_grade_on_track                                                     38
avg_class_size_high_school                                                     30
high_school_5_year_graduation_rate_total                                       25
principal_turnover_within_6_years                                              19
student_chronic_truancy_rate                                                    7
total_number_of_school_days                                                     7
student_attendance_rate                                                         7
high_school_4_ye

In [29]:
# cohort should be a category
hs_18['cohort'] = hs_18['cohort'].astype('object')
# check that again
list(hs_18.select_dtypes(['object', 'bool']))

['cohort',
 'rcdts',
 'school_name',
 'district',
 'city',
 'county',
 'district_type',
 'district_size',
 'school_type',
 'grades_served',
 'pupil_teacher_ratio_high_school',
 'teacher_avg_salary']

In [30]:
hs_18 = hs_18.drop(['teacher_avg_salary',
                    'principal_turnover_within_6_years'], axis=1).copy()

In [31]:
hs_18.isnull().sum().sort_values(ascending=False)

pupil_teacher_ratio_high_school                                               721
teacher_retention_rate                                                         67
percent_graduates_enrolled_in_a_postsecondary_institution_within_12_months     63
percent_graduates_enrolled_in_a_postsecondary_institution_within_16_months     63
percent_9th_grade_on_track                                                     38
avg_class_size_high_school                                                     30
high_school_5_year_graduation_rate_total                                       25
student_chronic_truancy_rate                                                    7
student_attendance_rate                                                         7
total_number_of_school_days                                                     7
high_school_4_year_graduation_rate_total                                        5
number_students_who_took_ap_classes_grade_12_total                              0
county          

In [32]:
zeros = hs_18.loc[:, ('high_school_4_year_graduation_rate_total',
                        'high_school_5_year_graduation_rate_total',
                        'percent_graduates_enrolled_in_a_postsecondary_institution_within_16_months',
                        'percent_graduates_enrolled_in_a_postsecondary_institution_within_12_months',
                        'percent_9th_grade_on_track')]

for zero in zeros:
    hs_18[zero].fillna(0, inplace=True)
    
hs_18.isnull().sum().sort_values(ascending=False)

pupil_teacher_ratio_high_school                                               721
teacher_retention_rate                                                         67
avg_class_size_high_school                                                     30
total_number_of_school_days                                                     7
student_attendance_rate                                                         7
student_chronic_truancy_rate                                                    7
number_students_who_took_ap_classes_grade_12_total                              0
district_size                                                                   0
percent_student_enrollment_white                                                0
grades_served                                                                   0
school_type                                                                     0
county                                                                          0
district_type   

In [33]:
nans = hs_18.loc[:, ('teacher_retention_rate', 
                    'pupil_teacher_ratio_high_school', 
                    'avg_class_size_high_school', 
                    'high_school_dropout_rate_total', 
                    'pupil_teacher_ratio_high_school',
                    'student_chronic_truancy_rate', 
                     'total_number_of_school_days', 
                     'student_attendance_rate')]

for nan in nans:
    hs_18[nan].fillna(hs_18[nan].mean(), inplace=True)

In [34]:
hs_18.isnull().sum().sort_values(ascending=False)

pupil_teacher_ratio_high_school                                               721
number_students_who_took_ap_classes_grade_12_total                              0
district_size                                                                   0
percent_student_enrollment_asian                                                0
percent_student_enrollment_hispanic_or_latino                                   0
percent_student_enrollment_black_or_african_american                            0
percent_student_enrollment_white                                                0
grades_served                                                                   0
school_type                                                                     0
district_type                                                                   0
percent_student_enrollment_american_indian_or_alaska_native                     0
county                                                                          0
city            

In [35]:
hs_18.shape

(721, 33)

In [36]:
hs_18.to_csv('../../data/val_set.csv', index=False)

I may almost be done cleaning, right?